# PageRank e Decomposição espectral

*Créditos: grande parte deste notebook foi traduzido de* https://github.com/fastai/numerical-linear-algebra

### Dois truques muito úteis

Aqui descrevemos duas ferramentas que vamos usar hoje, que são úteis de maneira geral.

1\. [Psutil](https://github.com/giampaolo/psutil) é uma ótima maneira de checar a utilização de memória. Isso vai ser útil pois iremos trabalhar com um dataset grande.

In [48]:
import psutil
import os
process = psutil.Process(os.getpid())
t = process.memory_info()
t.vms, t.rss

(5964115968, 1541984256)

In [49]:
def mem_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss * 1.0 / psutil.virtual_memory().total

In [50]:
mem_usage()

0.17951059341430664

2\. [TQDM](https://github.com/tqdm/tqdm) gives you progress bars.

In [51]:
from time import sleep
# Without TQDM
s = 0
for i in range(10):
    s += i
    sleep(0.2)
print(s)

45


In [52]:
# With TQDM
from tqdm import tqdm

s = 0
for i in tqdm(range(20)):
    s += i
    sleep(0.2)
print(s)

100%|██████████| 20/20 [00:04<00:00,  4.91it/s]

190


### Maneiras de pensar no SVD

- Compressão de dados
- SVD truncado permite substituir um conjunto grande de features por um conjunto pequeno de boas features

### SVD sob outras perspectivas

Nós normalmente descrevemos o SVD em termos de matrizes, $$A = U \Sigma V^\top,$$ mas também podemos descrevê-lo em termos de vetores.  SVD nos dá conjunto de vetores ortonormais ${v_j}$ e ${u_j}$ tais que $$ A v_j = \sigma_j u_j $$

onde $\sigma_j$ são escalares, chamados valores singulares.

**Q**: Isso nos lembra de alguma coisa?


**Resposta**

**Relação entre SVD e Decomposição Espectral**: os vetores singulares esquerdos de A são os autovetores de $AA^\top$. os vetores singulares direitos de A são os autovetores de $A^\top A$. Os valores singulares não-nulos de A são as raízes quadradas dos autovalores de $A^\top A$ (e $A A^\top$).

SVD é uma generalização da decomposição espectral. Nem todas as matrizes tem autovalores, mas TODAS tem valores singulares.

Vamos deixar o SVD de lado por um tempo e falar de como encontrar os autovalores de uma matriz definida positiva...

## Hoje: decomposição espectral

Os melhores métodos clássicos para computar o SVD são variantes de métodos para computar autovalores. Além da sua relação com o SVD, a decomposição espectral é útil por si só. Seguem abaixo alguns exemplos de aplicações da decomposição espectral:

- [calcular potências de matrizes rapidamente](http://www.onmyphd.com/?p=eigen.decomposition#h2_why)
- [n-ésimo número de Fibonacci](http://mathproofs.blogspot.com/2005/04/nth-term-of-fibonacci-sequence.html)
- Comportamento de EDOs
- Cadeias de Markov (Page Rank e muitos outro)
- [Linear Discriminant Analysis no dataset Iris](http://sebastianraschka.com/Articles/2014_python_lda.html)

"Eigenvalues are a way to see into the heart of a matrix... All the difficulties of matrices are swept away" - Gilbert Strang

**Teoremas relevantes:**

* Se A é simétrica, os autovalores de A são reais e $A=Q\Lambda Q^\top$, onde $Q$ é ortonormal.
* Se A é triangular, seus autovalores são...?

## Dataset DBpedia

Vamos começar com o **Power Method** (método da potência?), que encontra um autovetor. Talvez você esteja se perguntando *De que me serve um único autovetor?*  Na verdade, esta é a base do PageRank (leia [O autovetor de $25,000,000,000: a álgebra linear por trás do Google](http://www.rose-hulman.edu/~bryan/googleFinalVersionFixed.pdf) para maiores informações)

Em vez de tentar ranquear a importância de todos os websites da Internet, vamos usar um dataset de links do Wikipedia [DBpedia](http://wiki.dbpedia.org/). A DBpedia provê dados estruturados do Wikipedia de 125 idiomas.  

"*The full DBpedia data set features 38 million labels and abstracts in 125 different languages, 25.2 million links to images and 29.8 million links to external web pages; 80.9 million links to Wikipedia categories, and 41.2 million links to [YAGO](http://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/yago/) categories*" --[about DBpedia](http://wiki.dbpedia.org/about)

### Imports

In [53]:
import os, numpy as np, pickle
#from bz2 import BZ2File
#from datetime import datetime
#from pprint import pprint
from time import time
from tqdm import tqdm_notebook
from scipy import sparse

from sklearn.decomposition import randomized_svd
from sklearn.externals.joblib import Memory
from urllib.request import urlopen   # for python 2, use: from urllib2 import urlopen 

### Baixando os dados

Os dados que nós usaremos são:
- **redirects**: URLs que redirecionam para outras URLs
- **links**: que páginas apontam para quais páginas


In [54]:
PATH = 'data/dbpedia/'
URL_BASE = 'http://downloads.dbpedia.org/3.9/pt/'
filenames = ["redirects_pt.nt.bz2", "page_links_pt.nt.bz2"]

for filename in filenames:
    if not os.path.exists(PATH+filename):
        print("Downloading '%s', please wait..." % filename)
        open(PATH+filename, 'wb').write(urlopen(URL_BASE+filename).read())

In [55]:
redirects_filename = PATH+filenames[0]
page_links_filename = PATH+filenames[1]
redirects_filename

'data/dbpedia/redirects_pt.nt.bz2'

### Matriz de Adjacência do grafo

Nós iremos construir a **matriz de adjacência** do grafo, de que páginas apontam para quais páginas.

<img src="images/graph.png" alt="" style="width: 25%"/>
(source: [PageRank and HyperLink Induced Topic Search](https://www.slideshare.net/priyabrata232/page-rank-and-hyperlink))

<img src="images/adjaceny_matrix.png" alt="" style="width: 80%"/>
(source: [PageRank and HyperLink Induced Topic Search](https://www.slideshare.net/priyabrata232/page-rank-and-hyperlink))

*Q*: O que é que a potência $A^2$ nos diz?

Você poderá encontrar um exemplo aplicado a trechos de linhas aéreas em [worked out in these notes](http://www.utdallas.edu/~jwz120030/Teaching/PastCoursesUMBC/M221HS06/ProjectFiles/Adjacency.pdf).

Nós queremos armazenar que páginas apontam para quais páginas. Nós iremos armazenar essa informação em uma matriz quadrada, com um $1$ na posição $(r,c)$ indicando que o tópico na linha $r$ aponta para o tópico na coluna $c$. 

Você pode ler [mais sobre grafos aqui](http://www.geeksforgeeks.org/graph-and-its-representations/).

### Formato dos dados

Cada linha do arquivo redirect tem o seguinte formato
- `<http://pt.dbpedia.org/resource/Afonso_de_Espanha> <http://dbpedia.org/ontology/wikiPageRedirects> <http://pt.dbpedia.org/resource/Afonso_XIII_de_Espanha> .`

No slice abaixo, o mais 1, -1 servem para remover o <>

In [56]:
DBPEDIA_RESOURCE_PREFIX_LEN = len("http://pt.dbpedia.org/resource/")
SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)

In [57]:
SLICE

slice(32, -1, None)

In [58]:
# BUG? o arquivo nao esta lendo direito com BZ2File
def get_lines(filename): return (line.split() for line in BZ2File(filename) if line.count(' ') == 3)

In [59]:
def get_lines2(filename): return (line.split() for line in open(filename[:-4]) if line.count(' ') == 3)

Itere pelos redirections e crie um dicionário de origem para destino final.

In [60]:
def get_redirect(targ, redirects):
    seen = set()
    while True:
        transitive_targ = targ
        targ = redirects.get(targ)
        if targ is None or targ in seen: break
        seen.add(targ)
    return transitive_targ

In [61]:
def get_redirects(redirects_filename):
    redirects={}
    lines = get_lines2(redirects_filename)
    return {src[SLICE]:get_redirect(targ[SLICE], redirects) 
                for src,_,targ,_ in tqdm_notebook(lines, leave=False)}

In [62]:
redirects = get_redirects(redirects_filename)

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [63]:
redirects['Antraz']

'Carb\\u00FAnculo'

In [64]:
len(redirects)

605623

In [65]:
mem_usage()

0.1927952766418457

In [66]:
def add_item(lst, redirects, index_map, item):
    k = item[SLICE]
    lst.append(index_map.setdefault(redirects.get(k, k), len(index_map)))

In [67]:
limit=24000000 #24000000

In [88]:
# Computing the integer index map
index_map = dict() # links->IDs
lines = get_lines2(page_links_filename)
source, destination, data = [],[],[]
for l, split in tqdm_notebook(enumerate(lines), total=limit):
    if l >= limit: break
    add_item(source, redirects, index_map, split[0])
    add_item(destination, redirects, index_map, split[2])
    data.append(1)

HBox(children=(IntProgress(value=0, max=24000000), HTML(value='')))

In [69]:
len(index_map)

2467466

### Análise exploratória dos dados

Os passos abaixo são só para ilustrar que informação está em data e como ele está estruturada. Elas não são eficientes.

Vamos ver que tipo de itens estão no index_map:

In [22]:
page = 'Albino_Forjaz_de_Sampaio'
page_ind = index_map[page];
print(page,page_ind)

Albino_Forjaz_de_Sampaio 0


Vamos ver quantas vezes ele aparece como origem:

In [23]:
link_ind = [i for i,x in enumerate(source) if x == page_ind]
print(link_ind)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 7791909, 9946199]


Quais são os links de saída?

In [24]:
for ind in link_ind:
    print(source[ind],destination[ind])

0 1
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 11
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 7
0 1
0 26
0 27
0 28
0 29
0 30
0 0
0 0


Agora queremos checar uma das páginas de destino (aquela de índice $1$). Nota: normalmente você não acessa um dicionário procurando por seus valores. Isto é ineficiente e não é a forma como os dicionários são projetados para serem usados.

In [25]:
def printPagenameFromIndex(ind):
    for page_name, index in index_map.items():
        if index == ind:
            print(page_name)
            break

In [26]:
printPagenameFromIndex(1)

Lisboa


[Link para página em maio de 2015](https://pt.wikipedia.org/w/index.php?title=Albino_Forjaz_de_Sampaio&oldid=42207793)

Vamos encontrar o índice da página principal.

In [95]:
mainpage = b'Wikip\u00E9dia:P\u00E1gina_principal'.decode()
mainpage_ind = index_map[mainpage]; mainpage_ind

for ind in range(len(index_map)):
    source.append(ind)
    destination.append(mainpage_ind)
    data.append(0.1)
#     source.append(mainpage_ind)
#     destination.append(ind)
#     data.append(1)

### Criando a matriz

Nós iremos criar a seguir uma matriz esparsa usando o formato COO do Scipy, e então convertê-lo para CSR.

**Questão**: O que são COO e CSR?  Por que é que devemos criá-la com COO e convertê-la logo em seguida?

In [96]:
n = len(index_map); n
X = sparse.coo_matrix((data, (destination,source)), shape=(n,n), dtype=np.float32)
X = X.tocsr()

In [32]:
mem_usage()

0.2007579803466797

In [81]:
del(data,destination, source)

In [45]:
names = {i: name for name, i in index_map.items()}

In [29]:
mem_usage()

0.08244991302490234

### Salvar a matriz para não termos que recomputá-la

In [33]:
pickle.dump(X, open(PATH+'X.pkl', 'wb'))
pickle.dump(index_map, open(PATH+'index_map.pkl', 'wb'))

In [33]:
import pickle
import numpy as np
np.set_printoptions(precision=2)

PATH = 'data/dbpedia/'
X = pickle.load(open(PATH+'X.pkl', 'rb'))
index_map = pickle.load(open(PATH+'index_map.pkl', 'rb'))

## Power method (método da potência)

### Motivação

Uma matriz $n \times n$ é **diagonalizável** se ela possui $n$ autovetores linearmente independentes $v_1,\, \ldots v_n$.

Neste caso, qualquer vetor $w$ pode ser expresso como $w = \sum_{j=1}^n c_j v_j $, para alguns escalares $c_j$.

**Exercício:** Mostre que $$ A^k w = \sum_{j=1}^n c_j \lambda_j^k v_j$$

**Questão**: O que acontece quando $k$ é grande?

Esta é a ideia-chave do **power method**.

### Um exemplo pequeno

Vamos pegar as 10 primeiras linhas e colunas da matriz X para executar o método da potência. No entanto, o método não vai ser aplicado diretamente a esta submatriz. Estamos interessados na submatriz obtida ao se dividir cada coluna por sua soma.

In [34]:
a = X[:10,:10].copy().todense()
a /= a.sum(axis=0)
a

matrix([[0.14, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.21, 0.57, 0.  , 0.  , 0.  , 0.  , 0.  , 0.69, 0.  , 0.  ],
        [0.07, 0.  , 0.67, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.07, 0.  , 0.  , 0.  , 0.43, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.07, 0.  , 0.  , 0.  , 0.29, 0.  , 0.  , 0.  , 0.  , 0.  ],
        [0.07, 0.  , 0.33, 0.5 , 0.14, 0.  , 0.  , 0.  , 0.  , 0.33],
        [0.07, 0.  , 0.  , 0.  , 0.  , 0.  , 0.5 , 0.  , 0.  , 0.17],
        [0.14, 0.43, 0.  , 0.  , 0.14, 0.  , 0.25, 0.31, 0.  , 0.  ],
        [0.07, 0.  , 0.  , 0.  , 0.  , 0.  , 0.25, 0.  , 0.67, 0.  ],
        [0.07, 0.  , 0.  , 0.5 , 0.  , 1.  , 0.  , 0.  , 0.33, 0.5 ]],
       dtype=float32)

In [103]:
w = (np.ones((10,1))/10.0)
for i in range(100):
    w = a @ w
w /= np.sum(w)
print(w.T)

[[3.09e-86 6.11e-01 2.80e-19 8.82e-56 5.88e-56 1.35e-03 1.42e-03 3.81e-01
  1.23e-03 3.88e-03]]


### Tirando a prova real

**Nota:** Não se preocupe com a parte imaginária. Ela aparece pois alguns dos autovalores são imaginários.

In [104]:
Lambda, V = np.linalg.eig(a)
v = V[:,0] # autovetor associado ao maior autovalor em modulo
v /= np.sum(v)
print(v.T)

[[0.00e+00+0.j 6.16e-01+0.j 0.00e+00+0.j 0.00e+00+0.j 0.00e+00+0.j
  2.01e-16+0.j 3.83e-16+0.j 3.84e-01+0.j 3.83e-16+0.j 2.42e-16+0.j]]


### Parêntesis: Entendo a representação de matrizes esparsas

In [37]:
S = sparse.csr_matrix(np.array([[1,0,2],[3,4,5],[0,6,0]]))
Sr = S.sum(axis=0).A1; Sr

array([ 4, 10,  7], dtype=int64)

In [40]:
S

<3x3 sparse matrix of type '<type 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

[numpy.matrix.A1](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.A1.html#numpy.matrix.A1)

In [41]:
S.indices

array([0, 2, 0, 1, 2, 1], dtype=int32)

In [46]:
S.data

array([1, 2, 3, 4, 5, 6])

In [48]:
np.take(Sr, S.indices)

array([ 4,  7,  4, 10,  7, 10])

In [47]:
S.data * 1.0 / np.take(Sr, S.indice

array([ 0.25      ,  0.28571429,  0.75      ,  0.4       ,  0.71428571,
        0.6       ])

### Implementando o método da potência como função

In [38]:
def power_method_dense(a, max_iter=100):
    n = a.shape[0]
    a /= a.sum(axis=0) # divide pela soma das colunas, pois estamos usando a.T

    scores = np.ones((n,1), dtype=np.float32)/n  # chute inicial: todas mesmo ranking
    #scores = np.random.rand(n,1)                 # chute inicial: aleatorio
    for i in range(max_iter):
        scores = a @ scores
        nrm = np.linalg.norm(scores)
        scores /= nrm
        print(nrm)

    return scores.T

In [39]:
power_method_dense(a,10)

0.3735547
1.1439383
1.0389824
1.0085995
0.99967587
1.0001061
1.0025631
1.0055002
1.0080296
1.0100648


matrix([[7.70e-10, 6.43e-01, 4.29e-03, 1.77e-06, 1.18e-06, 1.89e-01,
         1.96e-01, 4.47e-01, 1.61e-01, 5.35e-01]], dtype=float32)

In [97]:
def power_method(A, max_iter=100):
    n = A.shape[1]
    A.data /= np.take(A.sum(axis=0).A1, A.indices)

    scores = np.ones((n,1), dtype=np.float32) * np.sqrt(A.sum()/(n*n)) # initial guess
    for i in range(max_iter):
        scores = A.dot(scores)
        nrm = np.linalg.norm(scores)
        scores /= nrm
        print(nrm)

    return scores.T

In [102]:
n = X.shape
A = X.copy()
z = power_method(A, max_iter=10)

1061.2333
1.076787
1.0333565
1.0273565
1.0221232
1.018504
1.0155473
1.0131183
1.0110149
1.0092705


In [93]:
def show_ex(v):
    print(', '.join(names[i] for i in np.abs(v.squeeze()).argsort()[-1:-10:-1]))

In [100]:
show_ex(z)

Wikip\u00E9dia:P\u00E1gina_principal, Anexo:Lista_de_asteroides, Wikip\u00E9dia:Artigo_sobre_evento_futuro, Estados_Unidos, Brasil, Anexo:Lista_de_presidentes_da_Bol\u00EDvia, Anexo:Lista_de_presidentes_da_Argentina, Anexo:Lista_de_Estados_soberanos, Portugal


**Questão**: Por que normalizar os scores a cada iteração?

**Decomposição espectral versus SVD:**

- SVD envolve duas bases, decomposição espectra envolve uma base
- Bases do SVD são ortonormais, base da decomposição espectral normalmente não é
- Todas as matrizes possuem um SVD, nem todas as matrizes (nem mesmo todas quadradas) tem uma decomposição espectral